In [3]:
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import sys

sys.path.append("../")

from src.common_functions import *

In [4]:
# sample data
# RAW_DATA_PATH1 = "../datasets/fraud/fraudTrain.csv"
# RAW_DATA_PATH2 = "../datasets/fraud/fraudTest.csv"
SAMPLED_DATA_PATH = "../datasets/fraud/cc_fraud_2.csv"
DATASET_NAME = Path(SAMPLED_DATA_PATH).stem
# raw_data1 = pd.read_csv("../datasets/fraud/fraudTrain.csv", index_col='Unnamed: 0')
# raw_data2 = pd.read_csv("../datasets/fraud/fraudTest.csv", index_col='Unnamed: 0')
# raw_data = pd.concat([raw_data1, raw_data2])
# raw_data = raw_data.rename(columns=dict(zip(raw_data.columns, list(map(to_snake_case, raw_data.columns)))))
# raw_data.head()
# _to_drop, df = train_test_split(raw_data, test_size=0.055, random_state=42, stratify=raw_data["is_fraud"])
# df.to_csv(SAMPLED_DATA_PATH, header=True, sep=",")
# del _to_drop
df = pd.read_csv(SAMPLED_DATA_PATH, index_col='Unnamed: 0')
print(df.shape[0])
df["is_fraud"].value_counts()/df.shape[0]*100

101882


is_fraud
0    99.478809
1     0.521191
Name: count, dtype: float64

In [5]:
df.isna().any()

trans_date_trans_time    False
cc_num                   False
merchant                 False
category                 False
amt                      False
first                    False
last                     False
gender                   False
street                   False
city                     False
state                    False
zip                      False
lat                      False
long                     False
city_pop                 False
job                      False
dob                      False
trans_num                False
unix_time                False
merch_lat                False
merch_long               False
is_fraud                 False
dtype: bool

In [10]:
df.value_counts()

trans_date_trans_time  cc_num            merchant                                 category       amt     first     last      gender  street                         city           state  zip    lat      long       city_pop  job                        dob         trans_num                         unix_time   merch_lat  merch_long   is_fraud
2020-12-31 23:58:34    3523843138706408  fraud_Prosacco, Kreiger and Kovacek      home           17.00   Grace     Williams  F       28812 Charles Mill Apt. 628    Plantersville  AL     36758  32.6176  -86.9475   1412      Drilling engineer          1970-11-20  6d04313bfe4b661b8ca2b6a499a320fe  1388534314  32.164145  -87.539669   0           1
2019-01-01 00:21:32    4334230547694630  fraud_Bruen-Yost                         misc_pos       6.85    Scott     Martin    M       7483 Navarro Flats             Freedom        WY     83120  43.0172  -111.0292  471       Education officer, museum  1967-08-02  f3c43d336e92a44fc2fb67058d5949e3  1325377292  43.75

Zeker nog eens nazien of je feature engineering kan verbeteren

In [18]:
COLUMNS_TO_DROP = ["city", "trans_date_trans_time", "cc_num", "merchant", "merchant", "first", "last", "zip", "trans_num", "dob"]
df_filtered = df.drop(COLUMNS_TO_DROP, axis=1)
numerical_columns = df_filtered.select_dtypes(
        include='number').columns.to_list()
df_filtered[]

category          14
amt            21119
gender             2
street           945
state             51
lat              931
long             932
city_pop         850
job              483
unix_time     101783
merch_lat     101560
merch_long    101755
is_fraud           2
dtype: int64

In [21]:
numerical_columns = df.select_dtypes(
        include='number').columns.to_list()
numerical_columns

['cc_num',
 'amt',
 'zip',
 'lat',
 'long',
 'city_pop',
 'unix_time',
 'merch_lat',
 'merch_long',
 'is_fraud']

In [15]:
y = "is_fraud"

# some generic cleaning

COLUMNS_TO_DROP = ["city", "trans_date_trans_time", "cc_num", "merchant", "merchant", "first", "last", "zip", "trans_num", "dob"]
assert df[y].isna().any() == False
raw_features = df.drop(COLUMNS_TO_DROP, axis=1)

# ML tuning

train, test = train_test_split(
    raw_features, test_size=0.2, random_state=0, stratify=df[y])

fit_le = LabelEncoder().fit(train[y])

In [16]:
param_grid = {
    'logistic__penalty': ['l2'],
    'logistic__C': [1, 10, 100, 1000],
    'pca__n_components': list(range(6, 32, 8)),
    'under__sampling_strategy': [0.1, 0.2, 0.3, 0.5, 0.7, 1],
    'over__sampling_strategy': [0.2, 0.3, 0.4, 0.5, 0.7, 1]
}

model = LogisticRegression()

MODEL_NAME = 'logistic'

grid = sklearn_gridsearch_using_pipeline(
    train, y_col_name=y, model=model, fit_le=fit_le, model_name='logistic', param_grid=param_grid)
best_pipeline_log_reg = grid.best_estimator_
best_pipeline_log_reg

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
write_pipeline(best_pipeline_log_reg, MODEL_NAME, DATASET_NAME)

In [ ]:
COLUMNS = ["under__sampling_strategy", "over__sampling_strategy", "pca__n_components", f"{MODEL_NAME}__C", f"{MODEL_NAME}__penalty", "roc_auc_mean", "roc_auc_std"]

pd.concat([
    pd.DataFrame(grid.cv_results_["params"]),
    pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["roc_auc_mean"]),
    pd.DataFrame(grid.cv_results_["std_test_score"], columns=["roc_auc_std"])
    ],axis=1).dropna(subset=["roc_auc_mean", "roc_auc_std"])[COLUMNS].sort_values(by=COLUMNS)

In [ ]:
evaluate_model(best_pipeline_log_reg, fit_le=fit_le, test=test, y_col_name=y)